In [22]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
cd '/content/drive/MyDrive/M2Datascale/Projet_Trans/BenchmarkLLM'

/content/drive/MyDrive/M2Datascale/Projet_Trans/BenchmarkLLM


In [24]:
import pandas as pd

In [1]:
!pip install einops

In [2]:
!pip install torch torchvision torchaudio

In [3]:
!pip install accelerate

In [4]:
!nvidia-smi

Sun Jan 14 09:45:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
#!pip uninstall -y transformers
#!pip install git+https://github.com/huggingface/transformers

In [6]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32, device_map="cuda",trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
data = pd.read_csv("./data.csv")

In [30]:
def microsoftQA(question,reponse_attendue):
  started_time = time.time()
  inputs = tokenizer(f'''{question}''', return_tensors="pt", return_attention_mask=False)
  outputs = model.generate(**inputs, max_length=200)
  text = tokenizer.batch_decode(outputs)[0]
  time_exec = time.time() - started_time
  new_row={'Model':'Microsoft-Phi2', 'Questions':question, 'Reponses':text, 'Temps':time_exec, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [31]:
list_data_microsoft=[]
for i in range(len(data)):
   list_data_microsoft.append(microsoftQA(data.iloc[i]['Question'],data.iloc[i]['Answers']))

In [32]:
list_data_microsoft

[{'Model': 'Microsoft-Phi2',
  'Questions': "What are the symptoms of Parkinson's Disease?",
  'Reponses': "What are the symptoms of Parkinson's Disease?\nThe symptoms of Parkinson's Disease are:\n- Tremor (shaking)\n- Rigidity (stiffness)\n- Bradykinesia (slowness of movement)\n- Postural instability (balance problems)\n- Speech and swallowing difficulties\n- Constipation\n- Sleep disturbances\n- Depression\n- Dementia\n- Fatigue\n- Loss of sense of smell\n- Loss of sense of taste\n- Loss of sense of touch\n- Loss of bladder control\n- Loss of sexual function\n- Loss of facial expression\n- Loss of automatic movements (e.g. blinking, smiling, swinging arms)\n- Loss of automatic postural adjustments (e.g. leaning forward when walking)\n- Loss of automatic movements of the tongue (e.g. chewing, swallowing)\n- Loss of automatic movements of the eyes (e.g. blinking, rolling eyes)\n- Loss of automatic",
  'Temps': 10.794450521469116,
  'Reponses_Attendues': 'Treatment includes medications 

In [ ]:
# started_time = time.time()
# inputs = tokenizer('''What are the symptoms of Parkinson's Disease?''', return_tensors="pt", return_attention_mask=False)
# outputs = model.generate(**inputs, max_length=200)
# text = tokenizer.batch_decode(outputs)[0]
# time_exec = time.time() - started_time
# #print(text)
# print(time_exec)

In [33]:
df_microsoft = pd.DataFrame(data=list_data_microsoft, index=None)

In [34]:
df_microsoft

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Microsoft-Phi2,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,10.794451,Treatment includes medications (levodopa) and ...
1,Microsoft-Phi2,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nA blood test is us...,9.852439,Treatment includes antiretroviral therapy (ART...
2,Microsoft-Phi2,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nDengue Feve...,9.471009,Diagnosis involves blood tests to detect the v...
3,Microsoft-Phi2,Can Glaucoma be prevented?,Can Glaucoma be prevented?\nGlaucoma cannot be...,10.606019,Preventing Glaucoma involves regular eye exams...
4,Microsoft-Phi2,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nThere is no k...,10.070481,Endometriosis is caused by the abnormal growth...
5,Microsoft-Phi2,How can I manage my Multiple Sclerosis?,How can I manage my Multiple Sclerosis?\nThe b...,9.137300,"Parkinson's Disease symptoms include tremors, ..."
6,Microsoft-Phi2,How is Cancer diagnosed?,How is Cancer diagnosed?\nCancer is diagnosed ...,10.350024,Symptoms vary depending on the type and stage ...
7,Microsoft-Phi2,What treatment options are available for Endom...,What treatment options are available for Endom...,9.884269,Managing Endometriosis involves pain managemen...
8,Microsoft-Phi2,How can I manage my Diabetes?,How can I manage my Diabetes?\nThe best way to...,9.285965,"Medication adherence, blood sugar monitoring."
9,Microsoft-Phi2,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nDengue fever i...,10.119051,"Lifestyle changes include avoiding alcohol, pr..."


In [35]:
df_falcon = pd.read_csv("./data_falcon.csv")

In [36]:
df_falcon

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Falcon-7b,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,3.680781,Treatment includes medications (levodopa) and ...
1,Falcon-7b,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nHIV/AIDS can be di...,4.535126,Treatment includes antiretroviral therapy (ART...
2,Falcon-7b,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nDengue feve...,8.151850,Diagnosis involves blood tests to detect the v...
3,Falcon-7b,Can Glaucoma be prevented?,Can Glaucoma be prevented?\nWhile there is no ...,2.783291,Preventing Glaucoma involves regular eye exams...
4,Falcon-7b,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nIt is difficu...,16.232186,Endometriosis is caused by the abnormal growth...
5,Falcon-7b,How can I manage my Multiple Sclerosis?,How can I manage my Multiple Sclerosis?\nThere...,5.455253,"Parkinson's Disease symptoms include tremors, ..."
6,Falcon-7b,How is Cancer diagnosed?,How is Cancer diagnosed?\nCancer is usually di...,4.216636,Symptoms vary depending on the type and stage ...
7,Falcon-7b,What treatment options are available for Endom...,What treatment options are available for Endom...,4.635307,Managing Endometriosis involves pain managemen...
8,Falcon-7b,How can I manage my Diabetes?,How can I manage my Diabetes?\nTo manage diabe...,4.537088,"Medication adherence, blood sugar monitoring."
9,Falcon-7b,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nDengue fever c...,4.925387,"Lifestyle changes include avoiding alcohol, pr..."


In [37]:
df = df_microsoft.append(df_falcon, ignore_index=True)

<ipython-input-37-1ea72f710404>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_microsoft.append(df_falcon, ignore_index=True)


In [38]:
df

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Microsoft-Phi2,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,10.794451,Treatment includes medications (levodopa) and ...
1,Microsoft-Phi2,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nA blood test is us...,9.852439,Treatment includes antiretroviral therapy (ART...
2,Microsoft-Phi2,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nDengue Feve...,9.471009,Diagnosis involves blood tests to detect the v...
3,Microsoft-Phi2,Can Glaucoma be prevented?,Can Glaucoma be prevented?\nGlaucoma cannot be...,10.606019,Preventing Glaucoma involves regular eye exams...
4,Microsoft-Phi2,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nThere is no k...,10.070481,Endometriosis is caused by the abnormal growth...
5,Microsoft-Phi2,How can I manage my Multiple Sclerosis?,How can I manage my Multiple Sclerosis?\nThe b...,9.137300,"Parkinson's Disease symptoms include tremors, ..."
6,Microsoft-Phi2,How is Cancer diagnosed?,How is Cancer diagnosed?\nCancer is diagnosed ...,10.350024,Symptoms vary depending on the type and stage ...
7,Microsoft-Phi2,What treatment options are available for Endom...,What treatment options are available for Endom...,9.884269,Managing Endometriosis involves pain managemen...
8,Microsoft-Phi2,How can I manage my Diabetes?,How can I manage my Diabetes?\nThe best way to...,9.285965,"Medication adherence, blood sugar monitoring."
9,Microsoft-Phi2,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nDengue fever i...,10.119051,"Lifestyle changes include avoiding alcohol, pr..."


In [39]:
df.to_csv("data_falcon_&_microsoftPhi2.csv", index=None)